In [9]:
import numpy as np
import pandas as pd
import flickrapi
from urllib.request import urlretrieve
from PIL import Image
from sklearn.metrics.pairwise import cosine_similarity
import auth

pd.set_option('display.max_columns', None)  
flickr = auth.connect()

# Set up the user preferences, location, duration, and budget
preferences = ['nature', 'beaches', 'mountains']
location = 'Paris'
duration = 5
budget = 1000

# Collect data from Flickr
photos = flickr.photos.search(tags=preferences, tag_mode='all', location=location, radius=10, extras="url_o, geo, tags, date, date_upload, date_taken, owner_name")

# Create a dataframe with the data collected from Flickr
df = pd.DataFrame(photos['photos']['photo'])



In [10]:
df

,id,owner,secret,server,farm,title,ispublic,isfriend,isfamily,dateupload,datetaken,datetakengranularity,datetakenunknown,ownername,tags,latitude,longitude,accuracy,context,url_o,height_o,width_o,place_id,woeid,geo_is_public,geo_is_contact,geo_is_friend,geo_is_family
0,52810737675,185212112@N03,c16a7dbf1b,65535,66,Lanzarote,1,0,0,1681244818,2023-03-09 12:44:30,0,0,maximilian_buhl,volcano landscape mountains vulkan berge vulka...,0,0,0,0,https://live.staticflickr.com/65535/5281073767...,2200.0,3200.0,NaN,NaN,NaN,NaN,NaN,NaN
1,52810668985,196322354@N04,84ee8d0bcd,65535,66,Yummy!,1,0,0,1681242009,2022-09-06 13:08:01,0,0,MathiasLiebing,italy dolomites southtyrol mountains food kais...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,52809508762,192175302@N07,3304a18567,65535,66,Summertime in England,1,0,0,1681236253,2023-04-11 17:44:34,0,1,evancelt,lego moc legomoc train trains barn cow dog tre...,0,0,0,0,https://live.staticflickr.com/65535/5280950876...,4000.0,6371.0,NaN,NaN,NaN,NaN,NaN,NaN
3,52809976341,66422453@N04,22dc7effc4,65535,66,It Was All Yellow,1,0,0,1681233584,2023-04-11 10:12:43,0,1,"www.35mmNegative.com(On a Break, Catchin",california wildflowers superbloom landscape ye...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,52810086634,34853729@N04,ff780e1892,65535,66,WestJet Link (opr by Pacific Coastal) SF34 CGPCF,1,0,0,1681231229,2022-11-17 10:51:14,0,0,Sandsman83,saab 340 turboprop westjet link pacific coasta...,0,0,0,0,https://live.staticflickr.com/65535/5281008663...,2160.0,3480.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,52795725860,73189284@N00,781f3e4c86,65535,66,Walking High,1,0,0,1680706115,2023-04-05 17:47:39,0,1,Pavlo Kuzyk,himalaya mountains people hiking nature landsc...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
96,52795172539,195964282@N07,c480bb3e86,65535,66,Daffodils and Mountains,1,0,0,1680693076,2023-04-03 17:24:16,0,0,Steve Don,nature landscape flora daffodils mountains lly...,0,0,0,0,https://live.staticflickr.com/65535/5279517253...,3648.0,5472.0,NaN,NaN,NaN,NaN,NaN,NaN
97,52794927609,61563441@N08,9319b0f37f,65535,66,Moody Lake - BC,1,0,0,1680682530,2022-04-30 21:09:10,0,0,GOR44Photographic@Gmail.com,britishcolumbia canada lake mono cloud landsca...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
98,52794851850,147446431@N04,9fc18c9447,65535,66,Spring Sunset in the Wallowa Mountains,1,0,0,1680670701,2023-04-04 19:13:24,0,0,Paul Freidel,rubypeak northeast eastern oregon mountains wa...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
df = df[['id', 'url_o', 'tags', 'longitude', 'latitude', 'owner', 'title', 'datetaken']]
df = df.dropna()

# Download the images and create feature vectors
features = []
for i, row in df.iterrows():
    url = row['url_o']
    filename = f"{row['id']}.jpg"
    urlretrieve(url, filename)
    img = Image.open(filename).resize((224, 224))
    feature = np.array(img).reshape(-1)
    features.append(feature)
df['features'] = features


In [16]:
df

,id,url_o,tags,longitude,latitude,owner,title,datetaken,features
0,52810737675,https://live.staticflickr.com/65535/5281073767...,volcano landscape mountains vulkan berge vulka...,0,0,185212112@N03,Lanzarote,2023-03-09 12:44:30,"[255, 255, 255, 255, 255, 255, 255, 255, 255, ..."
2,52809508762,https://live.staticflickr.com/65535/5280950876...,lego moc legomoc train trains barn cow dog tre...,0,0,192175302@N07,Summertime in England,2023-04-11 17:44:34,"[197, 232, 210, 196, 232, 209, 196, 231, 207, ..."
4,52810086634,https://live.staticflickr.com/65535/5281008663...,saab 340 turboprop westjet link pacific coasta...,0,0,34853729@N04,WestJet Link (opr by Pacific Coastal) SF34 CGPCF,2022-11-17 10:51:14,"[162, 175, 181, 162, 176, 181, 162, 175, 180, ..."
11,52808475418,https://live.staticflickr.com/65535/5280847541...,bleik andøya andøy beach strand stones steiner...,15.955773,69.273851,33231871@N08,White beach,2022-12-25 11:35:56,"[162, 211, 247, 164, 211, 247, 165, 212, 248, ..."
13,52808184630,https://live.staticflickr.com/65535/5280818463...,landscape landschaft mountains skyline lanzaro...,0,0,185212112@N03,Fuerteventura,2023-03-08 14:22:14,"[255, 255, 255, 255, 255, 255, 255, 255, 255, ..."
...,...,...,...,...,...,...,...,...,...
88,52797708503,https://live.staticflickr.com/65535/5279770850...,grüenboden gruonwald uri kantonuri alpen alps ...,0,0,183172956@N07,Grüen Boden / Gruonwald,2023-03-28 10:20:40,"[111, 98, 82, 111, 98, 82, 111, 98, 82, 111, 9..."
89,52796673762,https://live.staticflickr.com/65535/5279667376...,march evening sunset sky cloud sun sunsetcolou...,13.109833,47.802667,45239009@N04,"Sunset on the Gaisberg above Salzburg, Austria",2023-03-25 18:20:26,"[70, 62, 60, 66, 61, 60, 63, 60, 61, 69, 61, 6..."
90,52797267058,https://live.staticflickr.com/65535/5279726705...,skiing tahoe laketahoe sierraattahoe sierranev...,-120.085667,38.794688,45975345@N00,Freshies,2023-04-05 11:50:04,"[28, 55, 102, 27, 55, 102, 28, 55, 102, 29, 56..."
92,52796010854,https://live.staticflickr.com/65535/5279601085...,squarespace illux blue river ocean flow rock n...,0,0,194679789@N03,Frozen,2021-11-27 10:29:43,"[58, 64, 76, 58, 64, 76, 59, 65, 77, 61, 67, 7..."


In [29]:
df= df.reset_index(drop=True)
df.drop(['level_0', 'index'], axis = 1, inplace= True)
# Create a user profile
user_profile = np.zeros((1, len(df)))
for pref in preferences:
    indices = df[df['tags'].str.contains(pref)]['features'].index
    user_profile[0, indices] = 1

# Calculate the similarity between the user profile and the features of each image


In [36]:
df

,id,url_o,tags,longitude,latitude,owner,title,datetaken,features
0,52810737675,https://live.staticflickr.com/65535/5281073767...,volcano landscape mountains vulkan berge vulka...,0,0,185212112@N03,Lanzarote,2023-03-09 12:44:30,"[255, 255, 255, 255, 255, 255, 255, 255, 255, ..."
1,52809508762,https://live.staticflickr.com/65535/5280950876...,lego moc legomoc train trains barn cow dog tre...,0,0,192175302@N07,Summertime in England,2023-04-11 17:44:34,"[197, 232, 210, 196, 232, 209, 196, 231, 207, ..."
2,52810086634,https://live.staticflickr.com/65535/5281008663...,saab 340 turboprop westjet link pacific coasta...,0,0,34853729@N04,WestJet Link (opr by Pacific Coastal) SF34 CGPCF,2022-11-17 10:51:14,"[162, 175, 181, 162, 176, 181, 162, 175, 180, ..."
3,52808475418,https://live.staticflickr.com/65535/5280847541...,bleik andøya andøy beach strand stones steiner...,15.955773,69.273851,33231871@N08,White beach,2022-12-25 11:35:56,"[162, 211, 247, 164, 211, 247, 165, 212, 248, ..."
4,52808184630,https://live.staticflickr.com/65535/5280818463...,landscape landschaft mountains skyline lanzaro...,0,0,185212112@N03,Fuerteventura,2023-03-08 14:22:14,"[255, 255, 255, 255, 255, 255, 255, 255, 255, ..."
...,...,...,...,...,...,...,...,...,...
64,52797708503,https://live.staticflickr.com/65535/5279770850...,grüenboden gruonwald uri kantonuri alpen alps ...,0,0,183172956@N07,Grüen Boden / Gruonwald,2023-03-28 10:20:40,"[111, 98, 82, 111, 98, 82, 111, 98, 82, 111, 9..."
65,52796673762,https://live.staticflickr.com/65535/5279667376...,march evening sunset sky cloud sun sunsetcolou...,13.109833,47.802667,45239009@N04,"Sunset on the Gaisberg above Salzburg, Austria",2023-03-25 18:20:26,"[70, 62, 60, 66, 61, 60, 63, 60, 61, 69, 61, 6..."
66,52797267058,https://live.staticflickr.com/65535/5279726705...,skiing tahoe laketahoe sierraattahoe sierranev...,-120.085667,38.794688,45975345@N00,Freshies,2023-04-05 11:50:04,"[28, 55, 102, 27, 55, 102, 28, 55, 102, 29, 56..."
67,52796010854,https://live.staticflickr.com/65535/5279601085...,squarespace illux blue river ocean flow rock n...,0,0,194679789@N03,Frozen,2021-11-27 10:29:43,"[58, 64, 76, 58, 64, 76, 59, 65, 77, 61, 67, 7..."


In [51]:
user_profile.shape


array([[array([255, 255, 255, ..., 255, 255, 255], dtype=uint8),
        array([197, 232, 210, ..., 190, 225, 201], dtype=uint8),
        array([162, 175, 181, ..., 141, 142, 147], dtype=uint8),
        array([162, 211, 247, ..., 209, 238, 254], dtype=uint8),
        array([255, 255, 255, ..., 255, 255, 255], dtype=uint8),
        array([148, 191, 233, ..., 172, 209, 238], dtype=uint8),
        array([ 76, 106, 146, ...,  27,  40,  62], dtype=uint8),
        array([164, 155, 153, ..., 100, 133, 101], dtype=uint8),
        array([101,  83,  72, ..., 131, 136,  87], dtype=uint8),
        array([128, 137,  88, ..., 129, 135,  91], dtype=uint8),
        array([147, 136, 112, ..., 155, 151, 144], dtype=uint8),
        array([216, 228, 244, ...,  79,  77,  43], dtype=uint8),
        array([ 71,  80,  46, ..., 134, 141,  85], dtype=uint8),
        array([241, 246, 250, ..., 142, 135, 138], dtype=uint8),
        array([155, 139, 109, ..., 214, 209, 207], dtype=uint8),
        array([153, 123, 

In [52]:
similarities = cosine_similarity(user_profile,df['features'].values.reshape((1,69)))


ValueError: setting an array element with a sequence.

In [4]:
# Rank the images by similarity and filter based on budget and duration
df['similarity'] = similarities[0]
df = df[df['similarity'] > 0.5]
df = df[df['width'] / df['height'] >= 1]
df = df[df['width'] / df['height'] <= 1.5]
df = df[df['similarity'] > 0.5]
df = df[df['similarity'] > 0.5]

# Select the top images based on similarity and budget and duration constraints
df = df.sort_values(by='similarity', ascending=False)
df = df[df['width'] * df['height'] <= budget]
df = df[:duration]

# Print the recommended itinerary
print("Recommended itinerary:")
for i, row in df.iterrows():
    print(f"{i+1}. {row['url_o']}")

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0.]])